In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr

## Setup local dask cluster

In [ ]:
from datacube.utils.rio import configure_s3_access
from datacube.utils.dask import start_local_dask
import os
import dask

# configure dashboard link to go over proxy
dask.config.set({"distributed.dashboard.link":
                 os.environ.get('JUPYTERHUB_SERVICE_PREFIX', '/')+"proxy/{port}/status"});

# close previos client if any, so that one can re-run this cell without issues
client = locals().get('client', None)
if client is not None:
    client.close()
    del client
    
client = start_local_dask(n_workers=1,
                          threads_per_worker=4, 
                          mem_safety_margin='4G')
display(client)

# Configure GDAL for s3 access 
configure_s3_access(aws_unsigned=True,  # works only when reading public resources
                    client=client);

In [ ]:
from datacube import Datacube
import odc.algo

dc = Datacube()

In [ ]:
region_code, time = '55HGU', ('2019-06-01', '2019-08-31')
mode = 'tsmask' # fmask | tsmask | nomask
_roi = dict(x=np.s_[0:3000], y=np.s_[5000:8000])

dss = []
for p in ['ga_s2a_ard_nbar_granule', 'ga_s2b_ard_nbar_granule']:
    dss += dc.find_datasets(product=p, 
                            region_code=region_code, 
                            time=time)

tsm_dss = dc.find_datasets(product='s2_tsmask', 
                           time=time,
                           region_code=region_code
                           )
len(dss), len(tsm_dss)

## Do native load (lazy version with Dask)

In [ ]:
data_bands = [
 'nbar_blue',
 'nbar_green',
 'nbar_red',
]
mask_bands = ['fmask']
chunks = dict(x=1000, y=1000)

xx = dc.load(output_crs=dss[0].crs,
             resolution=(-10, 10),
             align=(0, 0),
             measurements=data_bands + mask_bands,
             group_by='solar_day',
             datasets=dss, 
             dask_chunks=chunks)

tsm = dc.load(product='s2_tsmask',
              like=xx.geobox,
              datasets=tsm_dss, 
              dask_chunks=chunks)
tsm

In [ ]:
# Select a smalled sub-section, to speed up testing
if _roi:
    xx = xx.isel(**_roi)
    tsm = tsm.isel(**_roi)

In [ ]:
xx_data = xx[data_bands]

fm_nocloud  = odc.algo.fmask_to_bool(xx.fmask, ('water', 'snow', 'valid'))
tsm_nocloud = odc.algo.fmask_to_bool(tsm.classification, ('valid',))

if mode == 'tsmask':
    nocloud = tsm_nocloud
elif mode == 'fmask':
    nocloud = fm_nocloud
elif mode == 'nomask':
    nocloud = None
else:
    raise ValueError(f'Expect mode to be one of fmask|tsmask|nomask')
    
if nocloud is not None:
    xx_clean = odc.algo.keep_good_only(xx_data, where=nocloud)
else:
    xx_clean = xx_data
    

In [ ]:
scale, offset = (1/10_000, 0)  # differs per product, aim for 0-1 values in float32
yy = odc.algo.int_geomedian(xx_clean, scale=scale, offset=offset)
rgba = odc.algo.to_rgba(yy, clamp=3000)
rgba.data

## Now we can run the computation

In [ ]:
%%time
rgba = rgba.compute()

## Display result

In [ ]:
import odc.ui
from IPython.display import Image

if max(rgba.shape) < 4000:
    display(Image(odc.ui.to_jpeg_data(rgba.data)))
else:
    print('image too large to show')

## Save COG

In [ ]:
from datacube.utils.cog import write_cog

fname = f"S2GM-{region_code}-{mode}-rgba.tif"
print(fname)
write_cog(rgba, fname,
          overwrite=True,
          overview_resampling='bilinear', 
          overview_levels=[2,4,8,16], 
          zlevel=4)

------------------------------------------------------------------